In [1]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from skimage import io

from data_generator import *
from dice_loss import *
from unet3d import *
from unet3d_dense import *
from resnet import *
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
    img_to_array,
    load_img
)

Using TensorFlow backend.


ModuleNotFoundError: No module named 'keras_frcnn'

In [ ]:
TRAIN_PATH = 'data/train/'
XTRAIN_PATH = f'{TRAIN_PATH}/image'
YTRAIN_PATH = f'{TRAIN_PATH}/label'

TEST_PATH = 'data/test/'
XTEST_PATH = f'{TEST_PATH}/image'
YTEST_PATH = f'{TEST_PATH}/label'

# STEPS=2000
STEPS = 50
EPOCHS=2
# EPOCHS = 1
MODEL_NAME = 'unet_lidc.hdf5'

## Data Generation and Augmentation

In [ ]:
image_generator_train = ImageDataGenerator(
    rotation_range=2,
    rescale=1./255,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.0
)

label_generator_train = ImageDataGenerator(
    rotation_range=2,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.0
)

image_generator_test = ImageDataGenerator(
    rescale=1./255,
    fill_mode='reflect',
    data_format='channels_last',
    validation_split=0.0
)

label_generator_test = ImageDataGenerator(
    # No rescale transform on target mask
    fill_mode='reflect',
    data_format='channels_last',
    validation_split=0.0
)

In [ ]:
train_generator = generator(
    TRAIN_PATH,
    image_generator_train,
    label_generator_train,
    batch_sz=1,
)
test_generator = generator(
    TEST_PATH,
    image_generator_test,
    label_generator_test, 
    batch_sz=1
)

In [ ]:
show_augmentation('data/train/image2/0.png', image_generator_train, n_rows=1)
show_augmentation('data/train/label2/0.png', label_generator_train, n_rows=1)

## Instantiate and Train the U-Net

In [ ]:
model = unet3d()
# model = unet3d_dense()
model.compile(
    optimizer = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199),
    loss=dice_coef_loss,
    metrics = [
        'accuracy',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.TruePositives()
    ]
)
model_checkpoint = ModelCheckpoint(MODEL_NAME, monitor='loss',verbose=1, save_best_only=True)

In [ ]:
model.fit_generator(
  train_generator,
  steps_per_epoch = 1000,
  epochs = 5
)

## Test the U-Net

### Training set predictions

In [ ]:
# results = model.predict(X_test, 2, verbose=1)

x0 = np.array(io.imread('data/train/image0/10.png', as_gray=True))
x1 = np.array(io.imread('data/train/image1/10.png', as_gray=True))
x2 = np.array(io.imread('data/train/image2/10.png', as_gray=True))
x3 = np.array(io.imread('data/train/image3/10.png', as_gray=True))

X = np.array([x0, x1, x2, x3])

y0 = np.array(io.imread('data/train/label0/10.png', as_gray=True))
y1 = np.array(io.imread('data/train/label1/10.png', as_gray=True))
y2 = np.array(io.imread('data/train/label2/10.png', as_gray=True))
y3 = np.array(io.imread('data/train/label3/10.png', as_gray=True))

Y = np.array([y0, y1, y2, y3])

Y_hat = model.predict(X.reshape(1, 256, 256, 4, 1))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

axs[0].imshow(X[2], cmap='bone')
axs[1].imshow(Y[2], cmap='bone')
axs[2].imshow(Y_hat[0].reshape(4, 256, 256)[2], cmap='bone')
plt.show()

### Test set predictions

In [ ]:
# results = model.predict(X_test, 2, verbose=1)

x0 = np.array(io.imread('data/test/image0/0.png', as_gray=True))
x1 = np.array(io.imread('data/test/image1/0.png', as_gray=True))
x2 = np.array(io.imread('data/test/image2/0.png', as_gray=True))
x3 = np.array(io.imread('data/test/image3/0.png', as_gray=True))

X = np.array([x0, x1, x2, x3])

y0 = np.array(io.imread('data/test/label0/0.png', as_gray=True))
y1 = np.array(io.imread('data/test/label1/0.png', as_gray=True))
y2 = np.array(io.imread('data/test/label2/0.png', as_gray=True))
y3 = np.array(io.imread('data/test/label3/0.png', as_gray=True))

Y = np.array([y0, y1, y2, y3])

Y_hat = model.predict(X.reshape(1, 256, 256, 4, 1))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

axs[0].imshow(X[2], cmap='bone')
axs[1].imshow(Y[2], cmap='bone')
axs[2].imshow(Y_hat[0].reshape(4, 256, 256)[2], cmap='bone')
plt.show()